In [ ]:
import os
import tempfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Model and performance evaluation
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
lbl = preprocessing.LabelEncoder()
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support as score
# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Crossroads/engg_data/crossroads_full_feature_set.csv")

In [ ]:
df.head()

,acct_id,acct_type_desc,event_name,event_date,plan_event_name,comp_name,section_name,row_name,SeatNum,price_code,PC1,Price,paid,add_datetime,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Start Year,LastYear,Term,TicketType,SeatUniqueID,Season,ClubExpYear,Tenure,UniqueID,isAttended,Resold,ResalePrice,ResaleDate,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Game Time,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag
0,117948790,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,1,FR,F,111.0,Y,2021-03-31 16:08:52,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-1,2021,NaN,25.0,CLT21HOU-101-10-1,Yes,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1
1,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,10,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-10,2021,NaN,20.0,CLT21HOU-101-10-10,No,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1
2,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,11,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-11,2021,NaN,20.0,CLT21HOU-101-10-11,No,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1
3,1489599590,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,12,FR,F,111.0,Y,2021-03-31 16:10:20,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-12,2021,NaN,4.0,CLT21HOU-101-10-12,Yes,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1
4,115101990,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,13,FR,F,111.0,Y,2021-03-31 16:08:21,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-13,2021,NaN,32.0,CLT21HOU-101-10-13,No,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1


In [ ]:
df.isnull().sum()

acct_id                           0
acct_type_desc                91418
event_name                        0
event_date                        0
plan_event_name              279172
comp_name                     80644
section_name                      0
row_name                          0
SeatNum                           0
price_code                        0
PC1                               0
Price                             0
paid                         154145
add_datetime                  80644
class_name                        0
status                            0
Sales_Source                1280704
isHost                            0
SeatType                          0
TicketClass                       0
Start Year                  1146325
LastYear                    1146325
Term                        1146445
TicketType                        0
SeatUniqueID                      0
Season                            0
ClubExpYear                 1146325
Tenure                      

In [ ]:
df["Sales_Source"].unique()

array([       nan, 1.0430e+04, 9.4900e+03, 9.4600e+03, 5.7760e+03,
       1.0450e+04, 5.4840e+03, 5.9440e+03, 1.0440e+04, 4.4440e+03,
       8.6640e+03, 1.0380e+04, 1.0424e+04, 4.2400e+03, 1.0470e+04,
       4.1500e+03, 4.3600e+03, 5.7840e+03, 1.0456e+04, 5.3600e+03,
       1.0140e+04, 1.0460e+04, 4.9600e+03, 4.3300e+03, 4.4100e+03,
       9.0000e+00, 1.0376e+04, 4.3480e+03, 1.0480e+04, 1.0580e+04,
       1.0510e+04, 3.7220e+03, 6.6500e+02, 1.0504e+04, 1.0190e+04,
       4.6160e+03, 9.3800e+03, 3.5280e+03, 1.0590e+04, 2.7600e+03,
       1.0500e+04, 4.7300e+03, 3.7980e+03, 1.0610e+04, 4.8500e+03,
       3.0380e+03, 1.0600e+04, 3.5520e+03, 1.0584e+04, 1.0490e+04,
       9.3000e+03, 7.4640e+03, 2.7760e+03, 1.0200e+04, 4.3200e+03,
       4.9240e+03, 4.2560e+03, 1.0570e+04, 9.0700e+03])

In [ ]:
df["isSTM"].unique()

array([nan,  1.])

In [ ]:
def combine_vals(row):
  if row["Resold"] in [1.0, '1', 'Yes']:
    return 1
  else:
    return 0

df["Resold"] = df.apply(lambda x: combine_vals(x), 1)

In [ ]:
df[["acct_type_desc"]] = df[["acct_type_desc"]].fillna("NA")
df[["acct_type_group"]] = df[["acct_type_group"]].fillna("NA")
df[["plan_event_name"]] = df[["plan_event_name"]].fillna("NA")
df[["plan_event_name_group"]] = df[["plan_event_name_group"]].fillna("NA")
df[["Sales_Source"]] = df[["Sales_Source"]].fillna(-1)
df[["Zone"]] = df[["Zone"]].fillna("NA")
df[["comp_name"]] = df[["comp_name"]].fillna("NA")
df[["comp_names_group"]] = df[["comp_names_group"]].fillna("NA")
df[["class_name_grp"]] = df[["class_name_grp"]].fillna("NA")
df[["paid"]] = df[["paid"]].fillna("NA")
df[["gameday_minus_start_year"]] = df[["gameday_minus_start_year"]].fillna(-1)
df[["lastyear_minus_gameday"]] = df[["lastyear_minus_gameday"]].fillna(-1000)
df[["unemployment_rate"]] = df[["unemployment_rate"]].fillna(-1)
df[["no_days_prebooked"]] = df[["no_days_prebooked"]].fillna(-1000)
df[["Term"]] = df[["Term"]].fillna(-1)
df[["Tenure"]] = df[["Tenure"]].fillna(-1)
df[["ResalePrice"]] = df[["ResalePrice"]].fillna(-1)
df[["isSTM"]] = df[["isSTM"]].fillna(0)
df["isSTM"] = df["isSTM"].astype(int)


In [ ]:
def pivot(train_data, group_cols, agg_col):
    data = train_data.groupby(group_cols).agg({agg_col: "count"}).reset_index()
    data = pd.pivot_table(data, values=agg_col, index=[group_cols[0]], columns=[group_cols[1]], aggfunc=np.sum).reset_index().rename_axis(None, axis=1)
    data = data.fillna(0)
    data["count"] = data["Yes"] + data["No"]
    data["%Attended"] = 100*data["Yes"]/(data["Yes"]+data["No"])
    return data

In [ ]:
accnt_type_df = pivot(df, ["acct_id", "isAttended"], "UniqueID")
accnt_type_df.head()

,acct_id,???,No,Yes,count,%Attended
0,190,6467.0,68700.0,369.0,69069.0,0.534248
1,14999990,0.0,8.0,0.0,8.0,0.000000
2,16999990,0.0,1.0,0.0,1.0,0.000000
3,110002390,14.0,29.0,97.0,126.0,76.984127
4,110003490,20.0,25.0,135.0,160.0,84.375000


In [ ]:
temp = df[df["isAttended"]!="???"]
acct_count_df = temp.groupby("acct_id").agg({"event_name": "nunique"}).reset_index()
acct_count_df = acct_count_df.rename(columns={"event_name": "event_name_count"})
acct_count_df.head()

,acct_id,event_name_count
0,190,18
1,14999990,2
2,16999990,1
3,110002390,18
4,110003490,16


In [ ]:
df = df.merge(accnt_type_df[["acct_id", "count", "%Attended"]], on="acct_id", how="left")
df = df.merge(acct_count_df[["acct_id", "event_name_count"]], on="acct_id", how="left")

In [ ]:
df["%Attended"] = df["%Attended"].fillna(-1)
df["event_name_count"] = df["event_name_count"].fillna(-1)

In [ ]:
df.head()

,acct_id,acct_type_desc,event_name,event_date,plan_event_name,comp_name,section_name,row_name,SeatNum,price_code,PC1,Price,paid,add_datetime,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Start Year,LastYear,Term,TicketType,SeatUniqueID,Season,ClubExpYear,Tenure,UniqueID,isAttended,Resold,ResalePrice,ResaleDate,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Game Time,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,count,%Attended,event_name_count
0,117948790,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,1,FR,F,111.0,Y,2021-03-31 16:08:52,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-1,2021,NaN,25.0,CLT21HOU-101-10-1,Yes,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,54.0,94.444444,18.0
1,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,10,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-10,2021,NaN,20.0,CLT21HOU-101-10-10,No,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,72.0,70.833333,18.0
2,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,11,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-11,2021,NaN,20.0,CLT21HOU-101-10-11,No,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,72.0,70.833333,18.0
3,1489599590,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,12,FR,F,111.0,Y,2021-03-31 16:10:20,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-12,2021,NaN,4.0,CLT21HOU-101-10-12,Yes,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,18.0,83.333333,18.0
4,115101990,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,13,FR,F,111.0,Y,2021-03-31 16:08:21,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-13,2021,NaN,32.0,CLT21HOU-101-10-13,No,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,144.0,72.222222,18.0


In [ ]:
df.isnull().sum()["%Attended"]

0

In [ ]:
t0 = df[(df["isAttended"]=="???") & (df["%Attended"]<21) & (df["count"]>=10) & (df["event_name_count"]>2)]
t0.shape, t0.acct_id.nunique()

((7128, 69), 96)

In [ ]:
t = df[df["isAttended"]=="???"]
t.shape, t.acct_id.nunique()

((128688, 69), 22102)

In [ ]:
100.0*7128/128688

5.538977993286087

In [ ]:
df["row_name"] = df["row_name"].astype(str)

In [ ]:
df.head()

,acct_id,acct_type_desc,event_name,event_date,plan_event_name,comp_name,section_name,row_name,SeatNum,price_code,PC1,Price,paid,add_datetime,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Start Year,LastYear,Term,TicketType,SeatUniqueID,Season,ClubExpYear,Tenure,UniqueID,isAttended,Resold,ResalePrice,ResaleDate,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Game Time,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,count,%Attended,event_name_count
0,117948790,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,1,FR,F,111.0,Y,2021-03-31 16:08:52,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-1,2021,NaN,25.0,CLT21HOU-101-10-1,Yes,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,54.0,94.444444,18.0
1,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,10,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-10,2021,NaN,20.0,CLT21HOU-101-10-10,No,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,72.0,70.833333,18.0
2,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,11,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-11,2021,NaN,20.0,CLT21HOU-101-10-11,No,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,72.0,70.833333,18.0
3,1489599590,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,12,FR,F,111.0,Y,2021-03-31 16:10:20,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-12,2021,NaN,4.0,CLT21HOU-101-10-12,Yes,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,18.0,83.333333,18.0
4,115101990,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,13,FR,F,111.0,Y,2021-03-31 16:08:21,OPEN,SOLD,-1.0,0,GA,Manifest,NaN,NaN,-1.0,Renewal,101-10-13,2021,NaN,32.0,CLT21HOU-101-10-13,No,0,-1.0,NaN,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,144.0,72.222222,18.0


In [ ]:
df = df.drop(["acct_id", "event_date", "SeatNum", "Game Time", 
              "add_datetime", "Start Year", "LastYear", "SeatUniqueID", "ResaleDate", 
              "ClubExpYear"], 1)

In [ ]:
df["Season"] = df["Season"].astype(str)

In [ ]:
def label_encode(row, col):
  if row[col]=="Yes":
    return 1
  elif row[col]=="No":
    return 0
  else:
    return row[col]

df["isAttended"] = df.apply(lambda x: label_encode(x, "isAttended"), 1)

In [ ]:
df.shape

(1286916, 59)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 589.10 MB
Memory usage after optimization is: 401.33 MB
Decreased by 31.9%


In [ ]:
cat_cols = []
for k,v in dict(df.dtypes).items():
  if v=="object":
    cat_cols.append(k)

cat_cols = list(set(cat_cols) - set(["event_name", "UniqueID", "isAttended"]))

from pandas.core.indexes import category
for col in cat_cols:
  df[col] = df[col].astype("category")

In [ ]:
# cat_cols

In [ ]:
# df = pd.get_dummies(df, columns = cat_cols, drop_first = True)

In [ ]:
att  = df["isAttended"].values
df = df.drop("isAttended", 1)
df["isAttended"] = att

In [ ]:
df.head()

,acct_type_desc,event_name,plan_event_name,comp_name,section_name,row_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,UniqueID,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,count,%Attended,event_name_count,isAttended
0,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,25.0,CLT21HOU-101-10-1,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,54.0,94.4375,18.0,1
1,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,20.0,CLT21HOU-101-10-10,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,72.0,70.8125,18.0,0
2,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,20.0,CLT21HOU-101-10-11,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,72.0,70.8125,18.0,0
3,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,4.0,CLT21HOU-101-10-12,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,18.0,83.3125,18.0,1
4,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,32.0,CLT21HOU-101-10-13,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,144.0,72.2500,18.0,0


In [ ]:
train_df = df[df["isAttended"]!="???"]
test_df = df[df["isAttended"]=="???"]

In [ ]:
val_data = pd.read_csv("/content/drive/MyDrive/Crossroads/Ensemble_Model/Ensemble_Validation_DataSet_v1")
val_data = val_data.drop("Unnamed: 0", 1)
val_df = train_df[train_df["UniqueID"].isin(val_data["UniqueID"].values.tolist())]

In [ ]:
val_df.head()

,acct_type_desc,event_name,plan_event_name,comp_name,section_name,row_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,UniqueID,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,count,%Attended,event_name_count,isAttended
4,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,32.0,CLT21HOU-101-10-13,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,144.0,72.250,18.0,0
5,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,32.0,CLT21HOU-101-10-14,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,144.0,72.250,18.0,1
8,Employee,CLT21HOU,21FS,Front Office/Club Staff,101,10,FR,F,0.0,Y,OPEN,COMP,-1.0,0,GA,Manifest,-1.0,Comp,2021,7.0,CLT21HOU-101-10-17,0,-1.0,0,G2,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_1,179.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,43.0,95.375,18.0,1
17,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,38.0,CLT21HOU-101-10-5,1,72.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,204.0,93.625,18.0,1
18,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,30.0,CLT21HOU-101-10-6,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,54.0,79.625,18.0,1


In [ ]:
train_df = train_df[~train_df["UniqueID"].isin(val_df["UniqueID"].values.tolist())]

In [ ]:
train_df.shape, val_df.shape

((926582, 59), (231646, 59))

In [ ]:
val_df.head()

,acct_type_desc,event_name,plan_event_name,comp_name,section_name,row_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,UniqueID,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,count,%Attended,event_name_count,isAttended
4,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,32.0,CLT21HOU-101-10-13,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,144.0,72.250,18.0,0
5,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,32.0,CLT21HOU-101-10-14,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,144.0,72.250,18.0,1
8,Employee,CLT21HOU,21FS,Front Office/Club Staff,101,10,FR,F,0.0,Y,OPEN,COMP,-1.0,0,GA,Manifest,-1.0,Comp,2021,7.0,CLT21HOU-101-10-17,0,-1.0,0,G2,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_1,179.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,43.0,95.375,18.0,1
17,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,38.0,CLT21HOU-101-10-5,1,72.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,204.0,93.625,18.0,1
18,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,30.0,CLT21HOU-101-10-6,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,54.0,79.625,18.0,1


In [ ]:
train_cnt = train_df["count"].values
train_att = train_df["%Attended"].values
train_event_cnt = train_df["event_name_count"].values

val_cnt = val_df["count"].values
val_att = val_df["%Attended"].values
val_event_cnt = val_df["event_name_count"].values

test_cnt = test_df["count"].values
test_att = test_df["%Attended"].values
test_event_cnt = test_df["event_name_count"].values

In [ ]:
train_df = train_df.drop(["count"], 1)
val_df = val_df.drop(["count"], 1)
test_df = test_df.drop(["count"], 1)

In [ ]:
train_df.head()

,acct_type_desc,event_name,plan_event_name,comp_name,section_name,row_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,UniqueID,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count,isAttended
0,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,25.0,CLT21HOU-101-10-1,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,94.4375,18.0,1
1,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,20.0,CLT21HOU-101-10-10,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,70.8125,18.0,0
2,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,20.0,CLT21HOU-101-10-11,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,70.8125,18.0,0
3,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,4.0,CLT21HOU-101-10-12,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,83.3125,18.0,1
6,Season,CLT21HOU,21FS,Not Comp,101,10,FR,F,111.0,Y,OPEN,SOLD,-1.0,0,GA,Manifest,-1.0,Renewal,2021,32.0,CLT21HOU-101-10-15,0,-1.0,0,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.375,-1.0,-1000.0,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.375,52.125,0.0,39.625,1,72.2500,18.0,0


In [ ]:
test_df["isAttended"].unique()

array(['???'], dtype=object)

In [ ]:
train_df.shape, val_df.shape, test_df.shape

((926582, 58), (231646, 58), (128688, 58))

In [ ]:
train_df["event_name"].nunique(), val_df["event_name"].nunique(), test_df["event_name"].nunique()

(18, 18, 2)

In [ ]:
del df

In [ ]:
import gc
gc.collect()

2661

In [ ]:
train_class_names = train_df["class_name"].values
val_class_names = val_df["class_name"].values
test_class_names = test_df["class_name"].values

In [ ]:
train_df = pd.get_dummies(train_df, columns = cat_cols, drop_first = True)
val_df = pd.get_dummies(val_df, columns = cat_cols, drop_first = True)
test_df = pd.get_dummies(test_df, columns = cat_cols, drop_first = True)

In [ ]:
y_train = train_df["isAttended"].values
y_val = val_df["isAttended"].values

train_df = train_df.drop("isAttended", 1)
val_df = val_df.drop("isAttended", 1)

In [ ]:
X_train = train_df.copy()
# y_train = train_df.iloc[:, -1].values
X_val = val_df.copy()
# y_val = val_df.iloc[:, -1].values
X_test = test_df.copy()
# y_test = test_df.iloc[:, -1].values

In [ ]:
# enc = TargetEncoder(cols=cat_cols, min_samples_leaf=20, smoothing=10).fit(X_train, y_train)
# X_train_encoded = enc.transform(X_train)
# X_val_encoded = enc.transform(X_val)
# X_test_encoded = enc.transform(X_test)

In [ ]:
X_val = X_val[X_train.columns.tolist()]
X_test = X_test[X_train.columns.tolist()]

In [ ]:
X_train.head()

,event_name,section_name,Price,Sales_Source,isHost,Term,Tenure,UniqueID,Resold,ResalePrice,isSTM,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,Week,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count,seat_segment_Low,seat_segment_Medium,row_name_group_G2,row_name_group_G3,row_name_group_G4,row_name_group_G5,SeatType_GA,status_COMP,status_HELD,status_SOLD,price_group_price_group_1,price_group_price_group_2,price_group_price_group_3,section_group_segment_Low,section_group_segment_Medium,TicketType_Broker,TicketType_Comp,TicketType_Employee,TicketType_Existing,TicketType_FlexPack,TicketType_Group,TicketType_Group Avail,TicketType_Held,TicketType_Internal Use,TicketType_Marketing,TicketType_New,TicketType_Obstructed Avail,TicketType_Opponents,TicketType_Other,TicketType_Other Avail,TicketType_Other SOLD,TicketType_Player,TicketType_Premium Avail,TicketType_Renewal,TicketType_SRO Avail,TicketType_Season Avail,TicketType_SingleGame,TicketType_SingleGm Avail,TicketType_Sponsor,TicketType_Upgrade,class_name_ACCMAN RELO,class_name_ACCOUNT MANAGER,class_name_ADA,class_name_ADA INF,class_name_BROADCAST,class_name_COACH,class_name_DDQ-BRDCSTHOLD,class_name_DIST-7HOLD,class_name_DIST-DHOLD,class_name_DIST-EXCP,class_name_DIST-OPEN,class_name_DIST-WIP,class_name_DOQ-BRDCSTHOLD,class_name_DOQ-WHEELCHAIR,class_name_EVENTS,class_name_GA_MINICAMP,class_name_GENTILE,class_name_GREG,class_name_GROUP,class_name_GROUP MANAGER,class_name_HANNAH,class_name_HUNTINGTON,class_name_INDIANA,class_name_JIM IRSAY,class_name_JODY,class_name_KILL SEATS,class_name_MEGAN,class_name_MISC,class_name_NO HOLD,class_name_OB2,class_name_OBSTRUCTED,class_name_OPEN,class_name_OWNERSHIP,class_name_PLAYERS,class_name_ROMINE,class_name_SEASON,class_name_SINGLE GAME HOLD,class_name_SINGLE GAME SRO,class_name_SPINZO1,class_name_SPINZO2,class_name_SPONSORS,class_name_TICKET OPS,class_name_UNTOUCHABLE,class_name_VISITING TEAM,class_name_VT SELL,class_name_WAIT WEB,class_name_XTEST,Season_2022,section_name_segment_Low,section_name_segment_Medium,row_segment_Low,row_segment_Medium,row_name_10,row_name_11,row_name_12,row_name_13,row_name_14,row_name_15,row_name_15N,row_name_16,row_name_16N,row_name_17,row_name_18,row_name_19,row_name_1N,row_name_2,row_name_20,row_name_21,row_name_21N,row_name_22,row_name_22N,row_name_23,row_name_23N,row_name_23W,row_name_24,row_name_24N,row_name_24W,row_name_25,row_name_25N,row_name_26,row_name_26N,row_name_26W,row_name_27,row_name_28,row_name_29,row_name_29N,row_name_2N,row_name_3,row_name_30,row_name_30N,row_name_31,row_name_31N,row_name_31W,row_name_32,row_name_33,row_name_34,row_name_34N,row_name_35,row_name_35N,row_name_36,row_name_36N,row_name_36W,row_name_3N,row_name_4,row_name_4N,row_name_5,row_name_5N,row_name_5W,row_name_6,row_name_6N,row_name_7,row_name_7N,row_name_8,row_name_9,row_name_B,row_name_C,row_name_SR1,row_name_SR2,seat_nums_group_G2,seat_nums_group_G3,seat_nums_group_G4,Zone_N,Zone_NA,Zone_S,Zone_W,comp_name_Club Alumni,comp_name_Coaches & Trainers,comp_name_Covid-19,comp_name_Customer Service,comp_name_Front Office/Club Staff,comp_name_Game Entertainment,comp_name_Game Official and Chain Crew,comp_name_League Employees,comp_name_Media,comp_name_Military,comp_name_Miscellaneous,comp_name_NA,comp_name_Not Comp,comp_name_Ownership Guests,comp_name_Players,comp_name_Promotions,comp_name_Sales Prospecting,comp_name_Sponsors,comp_name_Stadium Personnel,comp_name_VIP/Local Dignitaries,comp_name_Youth Football,comp_name_archiveBusiness,comp_names_group_G2,comp_names_group_G3,comp_names_group_G4,comp_names_group_NA,TicketType_group_group_1,TicketType_group_group_2,TicketType_group_group_3,TicketType_group_group_4,Game_Type_Reg,ticket_class_grp_ticket_class_2,ticket_class_grp_ticket_class_3,TicketClass_Infill,TicketClass_Manifest,TicketClass_Obstruct,TicketClass_Obstructed View,TicketClass_SRO,paid_NA,paid_P,paid_Y,we

In [ ]:
X_val.head()

,event_name,section_name,Price,Sales_Source,isHost,Term,Tenure,UniqueID,Resold,ResalePrice,isSTM,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,Week,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count,seat_segment_Low,seat_segment_Medium,row_name_group_G2,row_name_group_G3,row_name_group_G4,row_name_group_G5,SeatType_GA,status_COMP,status_HELD,status_SOLD,price_group_price_group_1,price_group_price_group_2,price_group_price_group_3,section_group_segment_Low,section_group_segment_Medium,TicketType_Broker,TicketType_Comp,TicketType_Employee,TicketType_Existing,TicketType_FlexPack,TicketType_Group,TicketType_Group Avail,TicketType_Held,TicketType_Internal Use,TicketType_Marketing,TicketType_New,TicketType_Obstructed Avail,TicketType_Opponents,TicketType_Other,TicketType_Other Avail,TicketType_Other SOLD,TicketType_Player,TicketType_Premium Avail,TicketType_Renewal,TicketType_SRO Avail,TicketType_Season Avail,TicketType_SingleGame,TicketType_SingleGm Avail,TicketType_Sponsor,TicketType_Upgrade,class_name_ACCMAN RELO,class_name_ACCOUNT MANAGER,class_name_ADA,class_name_ADA INF,class_name_BROADCAST,class_name_COACH,class_name_DDQ-BRDCSTHOLD,class_name_DIST-7HOLD,class_name_DIST-DHOLD,class_name_DIST-EXCP,class_name_DIST-OPEN,class_name_DIST-WIP,class_name_DOQ-BRDCSTHOLD,class_name_DOQ-WHEELCHAIR,class_name_EVENTS,class_name_GA_MINICAMP,class_name_GENTILE,class_name_GREG,class_name_GROUP,class_name_GROUP MANAGER,class_name_HANNAH,class_name_HUNTINGTON,class_name_INDIANA,class_name_JIM IRSAY,class_name_JODY,class_name_KILL SEATS,class_name_MEGAN,class_name_MISC,class_name_NO HOLD,class_name_OB2,class_name_OBSTRUCTED,class_name_OPEN,class_name_OWNERSHIP,class_name_PLAYERS,class_name_ROMINE,class_name_SEASON,class_name_SINGLE GAME HOLD,class_name_SINGLE GAME SRO,class_name_SPINZO1,class_name_SPINZO2,class_name_SPONSORS,class_name_TICKET OPS,class_name_UNTOUCHABLE,class_name_VISITING TEAM,class_name_VT SELL,class_name_WAIT WEB,class_name_XTEST,Season_2022,section_name_segment_Low,section_name_segment_Medium,row_segment_Low,row_segment_Medium,row_name_10,row_name_11,row_name_12,row_name_13,row_name_14,row_name_15,row_name_15N,row_name_16,row_name_16N,row_name_17,row_name_18,row_name_19,row_name_1N,row_name_2,row_name_20,row_name_21,row_name_21N,row_name_22,row_name_22N,row_name_23,row_name_23N,row_name_23W,row_name_24,row_name_24N,row_name_24W,row_name_25,row_name_25N,row_name_26,row_name_26N,row_name_26W,row_name_27,row_name_28,row_name_29,row_name_29N,row_name_2N,row_name_3,row_name_30,row_name_30N,row_name_31,row_name_31N,row_name_31W,row_name_32,row_name_33,row_name_34,row_name_34N,row_name_35,row_name_35N,row_name_36,row_name_36N,row_name_36W,row_name_3N,row_name_4,row_name_4N,row_name_5,row_name_5N,row_name_5W,row_name_6,row_name_6N,row_name_7,row_name_7N,row_name_8,row_name_9,row_name_B,row_name_C,row_name_SR1,row_name_SR2,seat_nums_group_G2,seat_nums_group_G3,seat_nums_group_G4,Zone_N,Zone_NA,Zone_S,Zone_W,comp_name_Club Alumni,comp_name_Coaches & Trainers,comp_name_Covid-19,comp_name_Customer Service,comp_name_Front Office/Club Staff,comp_name_Game Entertainment,comp_name_Game Official and Chain Crew,comp_name_League Employees,comp_name_Media,comp_name_Military,comp_name_Miscellaneous,comp_name_NA,comp_name_Not Comp,comp_name_Ownership Guests,comp_name_Players,comp_name_Promotions,comp_name_Sales Prospecting,comp_name_Sponsors,comp_name_Stadium Personnel,comp_name_VIP/Local Dignitaries,comp_name_Youth Football,comp_name_archiveBusiness,comp_names_group_G2,comp_names_group_G3,comp_names_group_G4,comp_names_group_NA,TicketType_group_group_1,TicketType_group_group_2,TicketType_group_group_3,TicketType_group_group_4,Game_Type_Reg,ticket_class_grp_ticket_class_2,ticket_class_grp_ticket_class_3,TicketClass_Infill,TicketClass_Manifest,TicketClass_Obstruct,TicketClass_Obstructed View,TicketClass_SRO,paid_NA,paid_P,paid_Y,we

In [ ]:
X_test.head()

,event_name,section_name,Price,Sales_Source,isHost,Term,Tenure,UniqueID,Resold,ResalePrice,isSTM,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,Week,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count,seat_segment_Low,seat_segment_Medium,row_name_group_G2,row_name_group_G3,row_name_group_G4,row_name_group_G5,SeatType_GA,status_COMP,status_HELD,status_SOLD,price_group_price_group_1,price_group_price_group_2,price_group_price_group_3,section_group_segment_Low,section_group_segment_Medium,TicketType_Broker,TicketType_Comp,TicketType_Employee,TicketType_Existing,TicketType_FlexPack,TicketType_Group,TicketType_Group Avail,TicketType_Held,TicketType_Internal Use,TicketType_Marketing,TicketType_New,TicketType_Obstructed Avail,TicketType_Opponents,TicketType_Other,TicketType_Other Avail,TicketType_Other SOLD,TicketType_Player,TicketType_Premium Avail,TicketType_Renewal,TicketType_SRO Avail,TicketType_Season Avail,TicketType_SingleGame,TicketType_SingleGm Avail,TicketType_Sponsor,TicketType_Upgrade,class_name_ACCMAN RELO,class_name_ACCOUNT MANAGER,class_name_ADA,class_name_ADA INF,class_name_BROADCAST,class_name_COACH,class_name_DDQ-BRDCSTHOLD,class_name_DIST-7HOLD,class_name_DIST-DHOLD,class_name_DIST-EXCP,class_name_DIST-OPEN,class_name_DIST-WIP,class_name_DOQ-BRDCSTHOLD,class_name_DOQ-WHEELCHAIR,class_name_EVENTS,class_name_GA_MINICAMP,class_name_GENTILE,class_name_GREG,class_name_GROUP,class_name_GROUP MANAGER,class_name_HANNAH,class_name_HUNTINGTON,class_name_INDIANA,class_name_JIM IRSAY,class_name_JODY,class_name_KILL SEATS,class_name_MEGAN,class_name_MISC,class_name_NO HOLD,class_name_OB2,class_name_OBSTRUCTED,class_name_OPEN,class_name_OWNERSHIP,class_name_PLAYERS,class_name_ROMINE,class_name_SEASON,class_name_SINGLE GAME HOLD,class_name_SINGLE GAME SRO,class_name_SPINZO1,class_name_SPINZO2,class_name_SPONSORS,class_name_TICKET OPS,class_name_UNTOUCHABLE,class_name_VISITING TEAM,class_name_VT SELL,class_name_WAIT WEB,class_name_XTEST,Season_2022,section_name_segment_Low,section_name_segment_Medium,row_segment_Low,row_segment_Medium,row_name_10,row_name_11,row_name_12,row_name_13,row_name_14,row_name_15,row_name_15N,row_name_16,row_name_16N,row_name_17,row_name_18,row_name_19,row_name_1N,row_name_2,row_name_20,row_name_21,row_name_21N,row_name_22,row_name_22N,row_name_23,row_name_23N,row_name_23W,row_name_24,row_name_24N,row_name_24W,row_name_25,row_name_25N,row_name_26,row_name_26N,row_name_26W,row_name_27,row_name_28,row_name_29,row_name_29N,row_name_2N,row_name_3,row_name_30,row_name_30N,row_name_31,row_name_31N,row_name_31W,row_name_32,row_name_33,row_name_34,row_name_34N,row_name_35,row_name_35N,row_name_36,row_name_36N,row_name_36W,row_name_3N,row_name_4,row_name_4N,row_name_5,row_name_5N,row_name_5W,row_name_6,row_name_6N,row_name_7,row_name_7N,row_name_8,row_name_9,row_name_B,row_name_C,row_name_SR1,row_name_SR2,seat_nums_group_G2,seat_nums_group_G3,seat_nums_group_G4,Zone_N,Zone_NA,Zone_S,Zone_W,comp_name_Club Alumni,comp_name_Coaches & Trainers,comp_name_Covid-19,comp_name_Customer Service,comp_name_Front Office/Club Staff,comp_name_Game Entertainment,comp_name_Game Official and Chain Crew,comp_name_League Employees,comp_name_Media,comp_name_Military,comp_name_Miscellaneous,comp_name_NA,comp_name_Not Comp,comp_name_Ownership Guests,comp_name_Players,comp_name_Promotions,comp_name_Sales Prospecting,comp_name_Sponsors,comp_name_Stadium Personnel,comp_name_VIP/Local Dignitaries,comp_name_Youth Football,comp_name_archiveBusiness,comp_names_group_G2,comp_names_group_G3,comp_names_group_G4,comp_names_group_NA,TicketType_group_group_1,TicketType_group_group_2,TicketType_group_group_3,TicketType_group_group_4,Game_Type_Reg,ticket_class_grp_ticket_class_2,ticket_class_grp_ticket_class_3,TicketClass_Infill,TicketClass_Manifest,TicketClass_Obstruct,TicketClass_Obstructed View,TicketClass_SRO,paid_NA,paid_P,paid_Y,we

In [ ]:
train_ids = X_train["UniqueID"].values
val_ids = X_val["UniqueID"].values
test_ids = X_test["UniqueID"].values

X_train = X_train.drop(["UniqueID", "event_name"], 1)
X_val = X_val.drop(["UniqueID", "event_name"], 1)
X_test = X_test.drop(["UniqueID", "event_name"], 1)

In [ ]:
X_train.head()

,section_name,Price,Sales_Source,isHost,Term,Tenure,Resold,ResalePrice,isSTM,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,Week,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count,seat_segment_Low,seat_segment_Medium,row_name_group_G2,row_name_group_G3,row_name_group_G4,row_name_group_G5,SeatType_GA,status_COMP,status_HELD,status_SOLD,price_group_price_group_1,price_group_price_group_2,price_group_price_group_3,section_group_segment_Low,section_group_segment_Medium,TicketType_Broker,TicketType_Comp,TicketType_Employee,TicketType_Existing,TicketType_FlexPack,TicketType_Group,TicketType_Group Avail,TicketType_Held,TicketType_Internal Use,TicketType_Marketing,TicketType_New,TicketType_Obstructed Avail,TicketType_Opponents,TicketType_Other,TicketType_Other Avail,TicketType_Other SOLD,TicketType_Player,TicketType_Premium Avail,TicketType_Renewal,TicketType_SRO Avail,TicketType_Season Avail,TicketType_SingleGame,TicketType_SingleGm Avail,TicketType_Sponsor,TicketType_Upgrade,class_name_ACCMAN RELO,class_name_ACCOUNT MANAGER,class_name_ADA,class_name_ADA INF,class_name_BROADCAST,class_name_COACH,class_name_DDQ-BRDCSTHOLD,class_name_DIST-7HOLD,class_name_DIST-DHOLD,class_name_DIST-EXCP,class_name_DIST-OPEN,class_name_DIST-WIP,class_name_DOQ-BRDCSTHOLD,class_name_DOQ-WHEELCHAIR,class_name_EVENTS,class_name_GA_MINICAMP,class_name_GENTILE,class_name_GREG,class_name_GROUP,class_name_GROUP MANAGER,class_name_HANNAH,class_name_HUNTINGTON,class_name_INDIANA,class_name_JIM IRSAY,class_name_JODY,class_name_KILL SEATS,class_name_MEGAN,class_name_MISC,class_name_NO HOLD,class_name_OB2,class_name_OBSTRUCTED,class_name_OPEN,class_name_OWNERSHIP,class_name_PLAYERS,class_name_ROMINE,class_name_SEASON,class_name_SINGLE GAME HOLD,class_name_SINGLE GAME SRO,class_name_SPINZO1,class_name_SPINZO2,class_name_SPONSORS,class_name_TICKET OPS,class_name_UNTOUCHABLE,class_name_VISITING TEAM,class_name_VT SELL,class_name_WAIT WEB,class_name_XTEST,Season_2022,section_name_segment_Low,section_name_segment_Medium,row_segment_Low,row_segment_Medium,row_name_10,row_name_11,row_name_12,row_name_13,row_name_14,row_name_15,row_name_15N,row_name_16,row_name_16N,row_name_17,row_name_18,row_name_19,row_name_1N,row_name_2,row_name_20,row_name_21,row_name_21N,row_name_22,row_name_22N,row_name_23,row_name_23N,row_name_23W,row_name_24,row_name_24N,row_name_24W,row_name_25,row_name_25N,row_name_26,row_name_26N,row_name_26W,row_name_27,row_name_28,row_name_29,row_name_29N,row_name_2N,row_name_3,row_name_30,row_name_30N,row_name_31,row_name_31N,row_name_31W,row_name_32,row_name_33,row_name_34,row_name_34N,row_name_35,row_name_35N,row_name_36,row_name_36N,row_name_36W,row_name_3N,row_name_4,row_name_4N,row_name_5,row_name_5N,row_name_5W,row_name_6,row_name_6N,row_name_7,row_name_7N,row_name_8,row_name_9,row_name_B,row_name_C,row_name_SR1,row_name_SR2,seat_nums_group_G2,seat_nums_group_G3,seat_nums_group_G4,Zone_N,Zone_NA,Zone_S,Zone_W,comp_name_Club Alumni,comp_name_Coaches & Trainers,comp_name_Covid-19,comp_name_Customer Service,comp_name_Front Office/Club Staff,comp_name_Game Entertainment,comp_name_Game Official and Chain Crew,comp_name_League Employees,comp_name_Media,comp_name_Military,comp_name_Miscellaneous,comp_name_NA,comp_name_Not Comp,comp_name_Ownership Guests,comp_name_Players,comp_name_Promotions,comp_name_Sales Prospecting,comp_name_Sponsors,comp_name_Stadium Personnel,comp_name_VIP/Local Dignitaries,comp_name_Youth Football,comp_name_archiveBusiness,comp_names_group_G2,comp_names_group_G3,comp_names_group_G4,comp_names_group_NA,TicketType_group_group_1,TicketType_group_group_2,TicketType_group_group_3,TicketType_group_group_4,Game_Type_Reg,ticket_class_grp_ticket_class_2,ticket_class_grp_ticket_class_3,TicketClass_Infill,TicketClass_Manifest,TicketClass_Obstruct,TicketClass_Obstructed View,TicketClass_SRO,paid_NA,paid_P,paid_Y,week_day_Saturday,week

In [ ]:
# df["isAttended"].value_counts()

In [ ]:
257283/900945, 900945/257283

(0.28557015134109187, 3.5017665372372058)

In [ ]:
# ! pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
# Now Catboost model training

model_catboost = CatBoostClassifier(objective="Logloss",
                                   iterations=400,
                                   learning_rate=0.1,
                                   depth=10,
                                   border_count=100,
                                   l2_leaf_reg=10,
                                   eval_metric='AUC',
                                   random_seed = 426,
                                   bagging_temperature=0.8,
                                   metric_period=50,
                                   early_stopping_rounds=30,
                                   task_type="GPU"
                                   )

model_catboost.fit(X_train, y_train,
                   eval_set=(X_val, y_val),
                   use_best_model=True,
                   verbose=50
                   )


0:	test: 0.8783390	best: 0.8783390 (0)	total: 69.9ms	remaining: 27.9s
50:	test: 0.8891717	best: 0.8891717 (50)	total: 1.96s	remaining: 13.4s
100:	test: 0.8906145	best: 0.8906145 (100)	total: 3.73s	remaining: 11s
150:	test: 0.8916442	best: 0.8916442 (150)	total: 5.8s	remaining: 9.57s
200:	test: 0.8923660	best: 0.8923660 (200)	total: 7.93s	remaining: 7.85s
250:	test: 0.8929718	best: 0.8929718 (250)	total: 9.83s	remaining: 5.83s
300:	test: 0.8935846	best: 0.8935846 (300)	total: 11.5s	remaining: 3.79s
350:	test: 0.8941644	best: 0.8941644 (350)	total: 13.2s	remaining: 1.85s
399:	test: 0.8946289	best: 0.8946289 (399)	total: 14.9s	remaining: 0us
bestTest = 0.8946288526
bestIteration = 399


In [ ]:
# evaluate the model using a given test condition
def evaluate_model(cv):
 # get the dataset
 # evaluate the model
 model_catboost = CatBoostClassifier(objective="Logloss",
                                   iterations=400,
                                   learning_rate=0.1,
                                   depth=10,
                                   border_count=100,
                                   l2_leaf_reg=10,
                                   eval_metric='AUC',
                                   random_seed = 426,
                                   bagging_temperature=0.8,
                                   metric_period=50,
                                   early_stopping_rounds=30,
                                   task_type="GPU"
                                   )
 scores = cross_val_score(model_catboost, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
 del model_catboost
 gc.collect()
 # return scores
 return np.mean(scores), scores.min(), scores.max()

In [ ]:
X_train.head(2)

,section_name,Price,Sales_Source,isHost,Term,Tenure,Resold,ResalePrice,isSTM,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,Week,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count,seat_segment_Low,seat_segment_Medium,row_name_group_G2,row_name_group_G3,row_name_group_G4,row_name_group_G5,SeatType_GA,status_COMP,status_HELD,status_SOLD,price_group_price_group_1,price_group_price_group_2,price_group_price_group_3,section_group_segment_Low,section_group_segment_Medium,TicketType_Broker,TicketType_Comp,TicketType_Employee,TicketType_Existing,TicketType_FlexPack,TicketType_Group,TicketType_Group Avail,TicketType_Held,TicketType_Internal Use,TicketType_Marketing,TicketType_New,TicketType_Obstructed Avail,TicketType_Opponents,TicketType_Other,TicketType_Other Avail,TicketType_Other SOLD,TicketType_Player,TicketType_Premium Avail,TicketType_Renewal,TicketType_SRO Avail,TicketType_Season Avail,TicketType_SingleGame,TicketType_SingleGm Avail,TicketType_Sponsor,TicketType_Upgrade,class_name_ACCMAN RELO,class_name_ACCOUNT MANAGER,class_name_ADA,class_name_ADA INF,class_name_BROADCAST,class_name_COACH,class_name_DDQ-BRDCSTHOLD,class_name_DIST-7HOLD,class_name_DIST-DHOLD,class_name_DIST-EXCP,class_name_DIST-OPEN,class_name_DIST-WIP,class_name_DOQ-BRDCSTHOLD,class_name_DOQ-WHEELCHAIR,class_name_EVENTS,class_name_GA_MINICAMP,class_name_GENTILE,class_name_GREG,class_name_GROUP,class_name_GROUP MANAGER,class_name_HANNAH,class_name_HUNTINGTON,class_name_INDIANA,class_name_JIM IRSAY,class_name_JODY,class_name_KILL SEATS,class_name_MEGAN,class_name_MISC,class_name_NO HOLD,class_name_OB2,class_name_OBSTRUCTED,class_name_OPEN,class_name_OWNERSHIP,class_name_PLAYERS,class_name_ROMINE,class_name_SEASON,class_name_SINGLE GAME HOLD,class_name_SINGLE GAME SRO,class_name_SPINZO1,class_name_SPINZO2,class_name_SPONSORS,class_name_TICKET OPS,class_name_UNTOUCHABLE,class_name_VISITING TEAM,class_name_VT SELL,class_name_WAIT WEB,class_name_XTEST,Season_2022,section_name_segment_Low,section_name_segment_Medium,row_segment_Low,row_segment_Medium,row_name_10,row_name_11,row_name_12,row_name_13,row_name_14,row_name_15,row_name_15N,row_name_16,row_name_16N,row_name_17,row_name_18,row_name_19,row_name_1N,row_name_2,row_name_20,row_name_21,row_name_21N,row_name_22,row_name_22N,row_name_23,row_name_23N,row_name_23W,row_name_24,row_name_24N,row_name_24W,row_name_25,row_name_25N,row_name_26,row_name_26N,row_name_26W,row_name_27,row_name_28,row_name_29,row_name_29N,row_name_2N,row_name_3,row_name_30,row_name_30N,row_name_31,row_name_31N,row_name_31W,row_name_32,row_name_33,row_name_34,row_name_34N,row_name_35,row_name_35N,row_name_36,row_name_36N,row_name_36W,row_name_3N,row_name_4,row_name_4N,row_name_5,row_name_5N,row_name_5W,row_name_6,row_name_6N,row_name_7,row_name_7N,row_name_8,row_name_9,row_name_B,row_name_C,row_name_SR1,row_name_SR2,seat_nums_group_G2,seat_nums_group_G3,seat_nums_group_G4,Zone_N,Zone_NA,Zone_S,Zone_W,comp_name_Club Alumni,comp_name_Coaches & Trainers,comp_name_Covid-19,comp_name_Customer Service,comp_name_Front Office/Club Staff,comp_name_Game Entertainment,comp_name_Game Official and Chain Crew,comp_name_League Employees,comp_name_Media,comp_name_Military,comp_name_Miscellaneous,comp_name_NA,comp_name_Not Comp,comp_name_Ownership Guests,comp_name_Players,comp_name_Promotions,comp_name_Sales Prospecting,comp_name_Sponsors,comp_name_Stadium Personnel,comp_name_VIP/Local Dignitaries,comp_name_Youth Football,comp_name_archiveBusiness,comp_names_group_G2,comp_names_group_G3,comp_names_group_G4,comp_names_group_NA,TicketType_group_group_1,TicketType_group_group_2,TicketType_group_group_3,TicketType_group_group_4,Game_Type_Reg,ticket_class_grp_ticket_class_2,ticket_class_grp_ticket_class_3,TicketClass_Infill,TicketClass_Manifest,TicketClass_Obstruct,TicketClass_Obstructed View,TicketClass_SRO,paid_NA,paid_P,paid_Y,week_day_Saturday,week

In [ ]:
importances = np.array(model_catboost.get_feature_importance())
sum_ = importances.sum()
imp_df = pd.DataFrame({'feature_importance': np.array(model_catboost.get_feature_importance()), 
              'feature_names': X_train.columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False).reset_index(drop=True)
def get_perc_imp(row):
  return 100.0*row["feature_importance"]/sum_

imp_df["Importance%"] = imp_df.apply(lambda x: get_perc_imp(x), 1)
imp_df = imp_df.sort_values("Importance%", ascending=False).reset_index(drop=True)

In [ ]:
imp_df

,feature_importance,feature_names,Importance%
0,5.323730e+01,%Attended,5.323730e+01
1,3.807773e+00,paid_NA,3.807773e+00
2,3.804700e+00,event_name_count,3.804700e+00
3,3.788633e+00,Game_Type_Reg,3.788633e+00
4,2.349376e+00,Week,2.349376e+00
5,1.906787e+00,section_name,1.906787e+00
6,1.660710e+00,no_days_prebooked,1.660710e+00
7,1.498517e+00,seat_segment_Low,1.498517e+00
8,1.470511e+00,plan_event_name_NA,1.470511e+00
9,1.410587e+00,Price,1.410587e+00


In [ ]:
y_pred_val_logits = model_catboost.predict_proba(X_val)
y_pred_val = [each[1] for each in y_pred_val_logits.tolist()]
y_pred_val = [1 if each>0.5 else 0 for each in y_pred_val]

In [ ]:
val_sub_df = pd.DataFrame({"UniqueID": val_ids, "CatBoost_model_preds": [each[1] for each in y_pred_val_logits]})

In [ ]:
val_sub_df.head()

,UniqueID,CatBoost_model_preds
0,CLT21HOU-101-10-13,0.709562
1,CLT21HOU-101-10-14,0.709562
2,CLT21HOU-101-10-17,0.967805
3,CLT21HOU-101-10-5,0.945482
4,CLT21HOU-101-10-6,0.828607


In [ ]:
from sklearn.metrics import classification_report
print(classification_report([int(each) for each in y_val], np.array(y_pred_val)))

              precision    recall  f1-score   support

           0       0.86      0.56      0.68     51457
           1       0.89      0.97      0.93    180189

    accuracy                           0.88    231646
   macro avg       0.87      0.77      0.80    231646
weighted avg       0.88      0.88      0.87    231646



In [ ]:
from sklearn.metrics import classification_report
print(classification_report([int(each) for each in y_val], np.array(y_pred_val)))

              precision    recall  f1-score   support

           0       0.86      0.56      0.68     51457
           1       0.89      0.97      0.93    180189

    accuracy                           0.88    231646
   macro avg       0.87      0.77      0.80    231646
weighted avg       0.88      0.88      0.87    231646



In [ ]:
val_pred_df = pd.DataFrame()
val_pred_df["UniqueID"] = val_ids
val_pred_df["count"] = val_cnt
val_pred_df["%Attended"] = val_att
val_pred_df["event_name_count"] = val_event_cnt
val_pred_df["class_name"] = val_class_names
val_pred_df["model_preds"] = y_pred_val

In [ ]:
def get_heuristic_pred(row):
  if ((row['%Attended']<50) & (row['count']>=10) & (row["event_name_count"]>2)) or \
  (row["class_name"] in ['DIST-7HOLD','DIST-EXCP','DIST-WIP','KILL SEATS','OB2','OWNERSHIP','SINGLE GAME SRO','XTEST']):
    return 0
  else:
    return row["model_preds"]

In [ ]:
val_pred_df["heuristic_pred"] = val_pred_df.apply(lambda x: get_heuristic_pred(x), 1)

In [ ]:
print(classification_report([int(each) for each in y_val], val_pred_df["heuristic_pred"]))

              precision    recall  f1-score   support

           0       0.83      0.57      0.68     51457
           1       0.89      0.97      0.93    180189

    accuracy                           0.88    231646
   macro avg       0.86      0.77      0.80    231646
weighted avg       0.88      0.88      0.87    231646



In [ ]:
X_test.shape

(128688, 1505)

In [ ]:
# X_test = X_test.drop("isAttended", 1)

In [ ]:
test_pred_logits = model_catboost.predict_proba(X_test)

In [ ]:
test_pred_logits[:5]

array([[0.03042903, 0.96957097],
       [0.12434916, 0.87565084],
       [0.12434916, 0.87565084],
       [0.07999063, 0.92000937],
       [0.12951841, 0.87048159]])

In [ ]:
test_preds = [each[1] for each in test_pred_logits.tolist()]
test_preds = [1 if each>0.5 else 0 for each in test_preds]

In [ ]:
sub_df = pd.DataFrame({"UniqueID": test_ids, "CatBoost_model_preds": ["Yes" if each==1 else "No" for each in test_preds]})

In [ ]:
sub_df.head()

,UniqueID,CatBoost_model_preds
0,CLT21LV-101-10-1,Yes
1,CLT21LV-101-10-10,Yes
2,CLT21LV-101-10-11,Yes
3,CLT21LV-101-10-12,Yes
4,CLT21LV-101-10-13,Yes


In [ ]:
sub_df["CatBoost_model_preds"].value_counts()

Yes    106114
No      22574
Name: CatBoost_model_preds, dtype: int64

In [ ]:
sub_df.shape

(128688, 2)

In [ ]:
sub_df.to_csv("/content/drive/MyDrive/Crossroads/predictions/Submission7_CatBoost.csv", index=False)

In [ ]:
import gc
gc.collect()

0

In [ ]:
sub_df.head()

,UniqueID,CatBoost_model_preds
0,CLT21LV-101-10-1,0.929422
1,CLT21LV-101-10-10,0.931482
2,CLT21LV-101-10-11,0.931482
3,CLT21LV-101-10-12,0.937883
4,CLT21LV-101-10-13,0.920768


In [ ]:
pred_df = pd.DataFrame()
pred_df["UniqueID"] = test_ids
# pred_df["acct_id_aa"] = ["AA"+str(each) for each in explicit_no_acc.tolist()]
pred_df["count"] = test_cnt
pred_df["%Attended"] = test_att
pred_df["event_name_count"] = test_event_cnt
pred_df["class_name"] = test_class_names
pred_df["model_preds"] = test_preds

In [ ]:
pred_df["heuristic_pred"] = pred_df.apply(lambda x: get_heuristic_pred(x), 1)

In [ ]:
pred_df.head()

,UniqueID,count,%Attended,event_name_count,class_name,model_preds,heuristic_pred
0,CLT21LV-101-10-1,54.0,94.4375,18.0,OPEN,1,1
1,CLT21LV-101-10-10,72.0,70.8125,18.0,OPEN,1,1
2,CLT21LV-101-10-11,72.0,70.8125,18.0,OPEN,1,1
3,CLT21LV-101-10-12,18.0,83.3125,18.0,OPEN,1,1
4,CLT21LV-101-10-13,144.0,72.2500,18.0,OPEN,1,1


In [ ]:
pred_df["heuristic_pred"].value_counts()

1    118116
0     10572
Name: heuristic_pred, dtype: int64

In [ ]:
val_sub_df.to_csv("/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Ayan_CatBoost_v1.2_val_preds.csv", index=False)


In [ ]:
sub_df.to_csv("/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Ayan_CatBoost_v1.2_test_preds.csv", index=False)


In [ ]:
val_sub_df["CatBoost_model_preds"].describe()

count    231646.000000
mean          0.778406
std           0.243748
min           0.000579
25%           0.782946
50%           0.874908
75%           0.912882
max           0.981993
Name: CatBoost_model_preds, dtype: float64

In [ ]:
sub_df["CatBoost_model_preds"].describe()

count    128688.000000
mean          0.836808
std           0.207667
min           0.000925
25%           0.853092
50%           0.895668
75%           0.926214
max           0.977900
Name: CatBoost_model_preds, dtype: float64

In [ ]:
len(test_preds) - sum(test_preds)

7888

In [ ]:
print(X_train.columns.tolist())

['section_name', 'Price', 'isHost', 'Term', 'Tenure', 'Resold', 'ResalePrice', 'isSTM', 'no_days_prebooked', 'gameday_minus_start_year', 'lastyear_minus_gameday', 'section_group', 'Week', 'Home Points', 'Away Points', 'unemployment_rate', 'covid_cases', 'Avg_Day_Temp', 'Precipitation', 'Avg_Dew_Point', 'weekend_flag', 'price_code_501', 'price_code_501E', 'price_code_501U', 'price_code_501W', 'price_code_502', 'price_code_503', 'price_code_503N', 'price_code_503W', 'price_code_504', 'price_code_504N', 'price_code_504W', 'price_code_505', 'price_code_506', 'price_code_507', 'price_code_507E', 'price_code_507N', 'price_code_507W', 'price_code_508', 'price_code_508E', 'price_code_508N', 'price_code_509', 'price_code_509N', 'price_code_509W', 'price_code_510', 'price_code_510N', 'price_code_511', 'price_code_511N', 'price_code_512', 'price_code_512N', 'price_code_513', 'price_code_513W', 'price_code_514', 'price_code_514N', 'price_code_515', 'price_code_516', 'price_code_517', 'price_code_5